In [9]:
# -*-coding:utf-8-*-
import pymysql
import os
import xlrd
# import pretty_errors

# 序时账里只取买入的数据

# connection = pymysql.connect(host='39.105.9.20', user='root', passwd='bigdata_oil',
#                 db='cxd_data', port=3306, charset='utf8')

connection = pymysql.connect(host='39.105.9.20', user='root', passwd='bigdata_oil',
                db='cxd_test', port=3306, charset='utf8')

cursor = connection.cursor()

oil_key = ['油', '沥青', '92', '95', '0', '35', '燃料', '油气', '苯', '热载体', '烯', '醚', '醇', '天然气', '液化',\
           '烷', '烃', '碳', 'MTBE', '剂', '脂', '酚', '酯', '原料']
com_key = ['公司', '加油', '厂', '气站', 'LTD', 'CO']
trade_key = ['购', '销']

docxlist = {}
com_name = ''

def name_filter(com_name): # 有些特殊的记账方式对应的名字
    filter = ['单位', '往来单位', '应付账款', '客户']
    com = com_name
    for f in filter:
        if com_name.startswith(f):
            com = com_name[len(f):]
    return com


def gci(filepath):
    # 遍历filepath下所有文件，包括子目录
    global com_name
    files = os.listdir(filepath)
    for fi in files:
        if fi == '.DS_Store':
            continue
        fi_d = os.path.join(filepath, fi)
        if os.path.isdir(fi_d):
            if '公司' in fi:
                com_name = fi
                docxlist[com_name] = []
            gci(fi_d)
        else:
            docxlist[com_name].append(fi_d)
    return docxlist


def read_xls(path):
    # xls文件解析
    record = []
    data = None
    try:
        data = xlrd.open_workbook(path, encoding_override="cp936")  # 打开xls文件
    except Exception as e:
        print(e)
        data = xlrd.open_workbook(path, encoding_override="cp1252")
    finally:
        if data.sheets():
            table = data.sheets()[0]  # 打开第一张表
            nrows = table.nrows  # 获取表的行数
            for i in range(nrows):  # 循环逐行打印
                if i == 0:  # 跳过第一行
                    continue
                record.append(table.row_values(i))
        return record


def data_merge(com, original_data, path):
    # 将序时账合并为对应的一条数据
    # 样例数据
    # [['10', '*', '', '', '2019-10-31', '记-1', '购货', '140587', '库存商品 - 0号  车用柴油(IV)', '', '', '', '', '',
    # 'RMB', '', 199821.24, 199821.24, 0.0, 'Manager', '', 'Manager', '', '', '', '32.221', '6,201.58'],
    # ['10', '*', '', '', '', '', '购货', '140564', '库存商品 - 0#车用柴油 （VI）', '', '', '', '', '', 'RMB', '',
    # 635065.12, 635065.12, 0.0, 'Manager', '', 'Manager', '', '', '', '100.654', '6,309.39'],
    # ['10', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '',
    # 108535.22, 108535.22, 0.0, 'Manager', '', 'Manager', '', '', '', '', ''],
    # ['10', '*', '', '', '', '', '购货', '220240', '应付账款 - 四川中油九洲北斗科技能源有限公司', '', '', '', '', '', 'RMB',
    # '', 943421.58, 0.0, 943421.58, 'Manager', '', 'Manager', '', '', '', '', '']]
    result = []
    oil_list = []
    com_list = []
    if len(original_data[0])<28: #文件结构错误
        return -1
    if '购' in original_data[0][6]:
        flag = 0    # 0:购 1:销
    else:
        flag = 1
    for record in original_data:
        if any(key in record[8] for key in oil_key) and all(key not in record[8] for key in com_key):
            # for r in record[8].replace(' ', '').split('-'):
            r = record[8].replace('库存商品 -', '').strip()
            if any(key in r for key in oil_key):
                oil_list.append(r)
        elif any(key in record[8] for key in com_key):
            for r in record[8].replace(' -- ', ' - ').replace(' ', '').split('-'):
                if any(key in r for key in com_key):
                    com_list.append(r)
        elif '账款' in record[8] and all(key not in record[8] for key in com_key):
            for r in record[8].replace(' -- ', ' - ').replace(' ', '').split('-'):
                if '账款' not in r and len(r)>1:
                    com_list.append(r)
    oil_num = len(oil_list)
    com_num = len(com_list)
    if len(oil_list) < 1 or len(com_list) < 1: # 对应解析失败
        if flag == 0:
            print('oil_num:',oil_num,oil_list)
            print('com_num:',com_num,com_list)
            print('公司名称解析失败')
            return -2
        else:
            print('此记录为卖出记录，不予统计')
            return 0
    elif oil_num == 1 and com_num == 1:   # 油品公司一一对应，最简单的匹配
        if flag == 0:
            xf = name_filter(com_list[0])
            gf = name_filter(com)
        else:
            print('此记录为卖出记录，不予统计')
            return 0
            # xf = com
            # gf = com_list[0]
        date = original_data[0][4]
        spmc = oil_list[0]
        Je = original_data[0][16]
        Sl = original_data[0][25] if original_data[0][25] else None
        Dj = original_data[0][26] if original_data[0][26] else None
        if ',' in str(Dj):
            Dj = Dj.replace(',', '')
        result.append((xf, gf, date, spmc, Je, Sl, Dj, path))
    elif oil_num > 1 and com_num == 1:  # 一个公司多个油品：
        if flag == 0:
            # xf = com_list[0]
            # gf = com
            xf = name_filter(com_list[0])
            gf = name_filter(com)
        else:
            print('此记录为卖出记录，不予统计')
            return 0
            # xf = com
            # gf = com_list[0]
        date = original_data[0][4]
        for m in oil_list:
            oil_ori = []
            for n in original_data:
                if m in n[8] and n[8] not in oil_ori:
                    spmc = m
                    oil_ori.append(n[8])
                    Je = n[16]
                    Sl = n[25] if n[25] else None
                    Dj = n[26] if n[26] else None
                    if ',' in str(Dj):
                        Dj = Dj.replace(',', '')
                    result.append((xf, gf, date, spmc, Je, Sl, Dj, path))
    elif oil_num > 1 and com_num > 1 and oil_num == com_num:  # 数量相同一一按顺序对应
        print('oil_num:',oil_num,oil_list)
        print('com_num:',com_num,com_list)
        if flag == 0:
            # xf = name_filter(com_list[0])
            gf = name_filter(com)
        else:
            print('此记录为卖出记录，不予统计')
            return 0
        date = original_data[0][4]
        price = []
        for n in original_data:
            # original_data: [['1', '*', '', '', '2019-01-31', '通用-1', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', \
            # 'RMB', '', 982758.62, 982758.62, 0.0, 'Manager', '', 'Manager', '', '', '', '200.00', '4,913.79', 2],
            # ['1', '*', '', '', '2019-01-31', '通用-1', '购货', '140506', '应付账款 - 四川中油九洲北斗科技能源有限公司', '', '', '', '', '', \
            # 'RMB', '', 982758.62, 982758.62, 0.0, 'Manager', '', 'Manager', '', '', '', '', '', 2]]
#             if m in n[8]:
#                 spmc = oil_list[0]
            if len(str(n[25])) > 2:
                Je = n[16]
                Sl = n[25] if n[25] else None
                Dj = n[26] if n[26] else None
                if ',' in str(Dj):
                    Dj = Dj.replace(',', '')
                price.append([Je, Sl, Dj, path])
        print('price:\n',price)
        print(len(price))
        title = []
        for i in range(len(com_list)):
            xf = com_list[i]
            spmc = oil_list[i]
            t = [xf, gf, date, spmc]
            title.append(t)
        print('title:\n',title)
        print(len(title))
        for i in range(len(price)):
            row = title[i] + price[i]
            result.append(tuple(row))
    else:   # 多个油品对应多家公司，油品数目与公司数目可能不等
        if flag == 0:
            # xf = name_filter(com_list[0])
            gf = name_filter(com)
        else:
            print('此记录为卖出记录，不予统计')
            return 0
        print('oil_num:',oil_num,oil_list)
        print('com_num:',com_num,com_list)
        print('合并记账，暂时没想到好方法,可先将此文件做记录，后续处理')
        return -2
    return result

def main():
    """
        序时账表头
        ['0期间', '1过账', '2审核', '3作废', '4日期', '5凭证字号', '6摘要', '7科目代码', '8科目名称',
        '9往来单位', '10部门', '11职员', '12自定义项目', '13业务编号', '14币别', '15汇率', '16原币金额',
        '17借方金额', '18贷方金额', '19制单人', '20审核人', '21过账人', '22结算方式', '23结算号',
         '24结算日期', '25数量', '26单价', '27附件数']
        """
    path_list = gci('2020第一批/XSZ测试')
    error_list = []
    error_data = []
    update_list = []
    match_error_list = []
    for k, v in path_list.items():
        for file_path in v:
            final_data = []
            if '序时' not in file_path:
                continue
            print(file_path)
            file_data = read_xls(file_path)

            if file_data:
                # 跳过无关公司的序时账
                if len(file_data[0]) < 27:
                    continue
                if '2019' in str(file_data[0][4]) or '2020' in str(file_data[0][4]):
                    temp = []
                    temp_data = []
                    index = 1
                    for r in file_data:
                        index += 1
                        r = r + [index, ]
                        if temp and r[4]:  # 判断是否是新账目(按日期）
#                             if any(key in temp[0][6] for key in trade_key):  # 判断是否是购销
                            if '购' in temp[0][6]:
                                if any(key in temp[0][8] for key in oil_key) and all(key not in temp[0][8] for key in com_key): # 是否包含油品
                                    temp_data.append(temp)
                            temp = []
                        temp.append(r)
                    for r in temp_data:
                        try:
                            middle_data = data_merge(k, r, file_path)
                            if middle_data == -2:
                                error_data.append([r, file_path])
                                match_error_list.append(file_path)  # 添加没想到好方法文件的路径信息(合并记账）
                                continue
                            elif middle_data == -1: # 解析失败企业
                                error_list.append(file_path)
                                continue
                            elif middle_data == 0: # 该记录为卖出记录
                                continue
                            if middle_data:
                                for j in middle_data:
                                    final_data.append(j)
                        except Exception as e:
                            print('有问题公司：', file_path)
                            print(str(e))
                            error_list.append(file_path)  # 添加解析文件出错的路径信息
            # print(final_data)
            update_list.extend(final_data)
    # print('err:',error_list)
    # print('update:',update_list)

    # 上传到数据库, 单独取录入错误的数据条数时要注释掉
    # 交易表入库sql
    trade_sql = "INSERT INTO financial_exchange_single (Xf_company_name,Gf_company_name,exchange_date,exchange_good,Je,Sl,Dj,source" \
                ") VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(trade_sql, update_list)
    connection.commit()

    no_match_list = {}
    for com in match_error_list:
        if com not in no_match_list.keys():
            no_match_list[com]=match_error_list.count(com)

    print('合并记账企业：\n', no_match_list)
    return error_list, error_data



def test(k, file_path): #test
    final_data = []
    file_data = read_xls(file_path)

    error_data = []
    match_error_list = []
    if len(file_data[0]) < 27:
        print('行数不足')
    if '2019' in str(file_data[0][4]) or '2020' in str(file_data[0][4]):
        temp = []
        temp_data = []
        index = 1
        for r in file_data:
            index += 1
            r = r + [index, ]
            if temp and r[4]:  # 判断是否是新账目(按日期）
                if any(key in temp[0][6] for key in trade_key):  # 判断是否是购销
                    if any(key in temp[0][8] for key in oil_key) and all(key not in temp[0][8] for key in com_key):  # 是否包含油品
                        temp_data.append(temp)
                temp = []
            temp.append(r)
        for r in temp_data:
            middle_data = data_merge(k, r, file_path)
            if middle_data == -2:
                error_data.append([r, file_path])
                match_error_list.append(file_path)  # 添加没想到好方法文件的路径信息(合并记账）
                continue
            elif middle_data == -1: # 解析失败企业
                error_list.append(file_path)
                continue
            elif middle_data == 0: # 该记录为卖出记录
                continue
            if middle_data:
                for j in middle_data:
                    final_data.append(j)
    print(error_data)
    print('match_error_list:\n')
    print(match_error_list)
    print('final_data:\n')
    print(final_data)


if __name__ == "__main__":
    e1, e2 = main()
    print(e1)
    print(e2)
    
#     with open('./files/序时账解析错误公司.txt', 'w', encoding='utf-8') as w:
#         for l in e1:
#             w.write(l+'\n')
#     # print('error_data: \n', e2[0][0])
#     with open('./files/序时账合并记账详细企业记录.txt', 'w', encoding='utf-8') as w:
#         for l in e2:
#             for err in l[0]:
#                 # for e in err:
#                 error_row_index = ' index: ' + str(err[-1])
#                 w.write(l[1] + error_row_index + '\n')

2020第一批/XSZ测试\大连港兴联合石化有限公司\59e1a461-dfc8-459c-acf6-d887b8fb7199_会计分录序时簿大连港兴联合石化有限公司.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
oil_num: 6 ['柴油', '0#', '燃料油', '120#', '燃料油', '180#']
com_num: 6 ['辽宁鸿鑫百度船舶燃料油有限公司', '盘锦金力源化工有限公司', '盘锦旺海化工有限公司', '山东吉七能源有限公司', '营口港船舶燃料供应有限责任公司', '淄博鲁临化工有限公司']
price:
 [[1385931.05, '225.50', '6146.04', '2020第一批/XSZ测试\\大连港兴联合石化有限公司\\59e1a461-dfc8-459c-acf6-d887b8fb7199_会计分录序时簿大连港兴联合石化有限公司.xls'], [1086206.89, '300.00', '3620.69', '2020第一批/XSZ测试\\大连港兴联合石化有限公司\\59e1a461-dfc8-459c-acf6-d887b8fb7199_会计分录序时簿大连港兴联合石化有限公司.xls'], [24685636.11, '7,053.53', '3499.76', '2020第一批/XSZ测试\\大连港兴联合石化有限公司\\59e1a461-dfc8-459c-acf6-d887b8fb7199_会计分录序时簿大连港兴联合石化有限公司.xls']]
3
title:
 [['辽宁鸿鑫百度船舶燃料油有限公司', '大连港兴联合石化有限公司', '2019-01-31', '柴油'], ['盘锦金力源化工有限公司', '大连港兴联合石化有限公司', '2019-01-31', '0#'], ['盘锦旺海化工有限公司', '大连港兴联合石化有限公司', '2019-01-31', '燃料油'], ['山东吉七能源有限公司', '大连港兴联合石化有限公司', '2019-01-31', '120#'], ['营口港船舶燃料供应有限责任公司', '大连港兴联合石化有限公司', '2019-01-31', '燃料油

D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 3")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 6")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 9")
  result = self._query(query)


In [ ]:
file_path = r'2020第一批/XSZ\2020-03-04\中油鸿阳（大连）石油化工有限公司\426a23c3-dc41-4eb5-94da-ab3fcbc97a2b_中油鸿阳（大连）石油化工有限公司1-12月序时账.xls'
com = '中油鸿阳（大连）石油化工有限公司'
test(com,file_path)

In [21]:
oil_key = ['油', '沥青', '92', '95', '0', '35', '10']
com_key = ['公司','加油', '厂']

record = '盘锦辽河油田大力能源有限油'
print(any(key in record for key in oil_key))
print(all(key not in record for key in com_key))
if any(key in record for key in oil_key) and all(key not in record for key in com_key):
    print(record)
else:
    print('not')

True
True
盘锦辽河油田大力能源有限油


In [9]:
test = '库存商品 - 汽油*车用汽油（VIA）92#'
r = test.replace(' -- ', ' - ').replace(' ', '').split('-')
r

['库存商品', '汽油*车用汽油（VIA）92#']

In [8]:
test = '库存商品 - 汽油*车用汽油（VIA）92#'
r = test.replace('库存商品 -', '').strip()
# r = r.strip()
r

'汽油*车用汽油（VIA）92#'

In [39]:
docxlist = {}
com_name = ''

path_dict = gci('2020第一批/XSZ')
multi_list = []
# print(type(path_list))
for k in path_dict:
    com_list = path_dict[k]
    if len(com_list)>1:
#         print(com_list)
        multi_list.append(com_list)
print(len(multi_list))
print(multi_list)

109
[['2020第一批/XSZ\\2020-03-04\\中化海洋能源（大连）有限公司\\4dbf6f1a-0195-4bf9-922d-5d3218c1dcb1_序时账中化海洋能源（大连）有限公司.xls', '2020第一批/XSZ\\2020-03-04\\中化海洋能源（大连）有限公司\\770fb7ad-69b7-4694-b47f-ca86aa834887_序时账中化海洋能源（大连）有限公司.xls'], ['2020第一批/XSZ\\2020-03-04\\中油东成石油化工（大连）有限公司\\578b0c03-1c41-4779-aad2-f5d3559360d4_序时账中油东成石油化工（大连）有限公司.xls', '2020第一批/XSZ\\2020-03-04\\中油东成石油化工（大连）有限公司\\a021b467-b936-4161-801b-7bef67f9a970_序时账中油东成石油化工（大连）有限公司.xls'], ['2020第一批/XSZ\\2020-03-04\\中油凯胜源（大连）石油化工有限公司\\47f69179-86e2-4f23-9c47-4306fd65bdc4_中油凯胜源（大连）石油化工有限公司序时账.xls', '2020第一批/XSZ\\2020-03-04\\中油凯胜源（大连）石油化工有限公司\\7d7fbe32-8fb8-4f51-af62-a3df0886b822_中油凯胜源（大连）石油化工有限公司序时账.xls', '2020第一批/XSZ\\2020-03-04\\中油凯胜源（大连）石油化工有限公司\\c6868a70-8677-4de3-9839-16cab34da5f7_中油凯胜源（大连）石油化工有限公司数量金额明细账.xls'], ['2020第一批/XSZ\\2020-03-04\\中油加仑多石化（大连）有限公司\\0888a448-2b5d-4e40-a213-14330e5d4539_序时账中油加仑多石化（大连）有限公司.xls', '2020第一批/XSZ\\2020-03-04\\中油加仑多石化（大连）有限公司\\3d071c05-74c3-41a0-a712-3e4e403a27f6_序时账中油加仑多石化（大连）有限公司.xls'], ['2020第一批/XSZ\\2020-03-04\

In [40]:
with open('./files/复数序时账公司.txt', 'w', encoding='utf-8') as w:
    for l in multi_list:
        w.write(l[0]+'\n') 

In [38]:
docxlist = {}
com_name = ''

def gci_name(filepath):
    # 遍历filepath下所有文件，包括子目录
    global com_name
    files = os.listdir(filepath)
    for fi in files:
        if fi == '.DS_Store':
            continue
        fi_d = os.path.join(filepath, fi)
        if os.path.isdir(fi_d):
            if '公司' in fi:
                com_name = fi
                docxlist[com_name] = []
            gci(fi_d)
        else:
            docxlist[com_name].append(fi_d)
    return docxlist

path_dict = gci_name('2020第一批/问题已修改企业')
merge_list = []
print(len(path_dict))
for k in path_dict:
    merge_list.append(k)
    
print(len(merge_list))
print(merge_list)
    
with open('./files/合并记账公司.txt', 'w', encoding='utf-8') as w:
    for l in merge_list:
        w.write(l+'\n')

137
137
['中化龙华国际贸易（大连）有限公司', '中植国能石油化工（大连）有限公司', '中油(大连)石油天然气有限公司', '中油万邦石油化工（大连）有限公司', '中油东成石油化工（大连）有限公司', '中油东港（大连）能源有限公司', '中油众恒石化(大连)有限公司', '中油冠东（大连）石化有限公司', '中油凯胜源（大连）石油化工有限公司', '中油创胜（大连）石油化工有限公司', '中油北部湾能源（大连）有限公司', '中油千溪能源（大连）有限责任公司', '中油华新伟业（大连）石油化工有限公司', '中油华海石油化工（大连）有限公司', '中油华远石油化工（大连）有限公司', '中油博海石油化工（大连）有限公司', '中油天宏能源（大连）有限公司', '中油天禹（大连）石油化工有限公司', '中油天莹能源（大连）有限公司', '中油富兴石油化工（大连）有限公司', '中油志明（大连）能源有限公司', '中油恒启石化（大连）有限责任公司', '中油新胜能源（大连）有限公司', '中油景顺石化（大连）有限公司', '中油森强石油化工（大连）有限公司', '中油楚骐石油化工（大连）有限公司', '中油正韬能源（大连）有限公司', '中油永顺石化（大连）有限公司', '中油泰达（大连）石油化工有限公司', '中油海元（大连）能源有限公司', '中油海涛石油化工（大连）有限公司', '中油海航石油化工（大连）有限公司', '中油润添（大连）能源有限公司', '中油盎通石油化工（大连）有限公司', '中油盖华（大连）石油化工有限公司', '中油睿博能源（大连）有限公司', '中油腾达石油化工（大连）有限公司', '中油运达（大连）石油化工有限公司', '中油运通（大连）石油化工有限公司', '中油远翔石油化工（大连）有限公司', '中油金成石油化工（大连）有限公司', '中油鑫峰能源（大连）有限公司', '中油鑫洋（大连）石油化工有限公司', '中油鑫禹石油化工（大连）有限公司', '中油长恒石油化工（大连）有限公司', '中油顺通石油化工（大连）有限公司', '中油飞宇龙石油（大连）有限公司', '中油鸿阳（大连）石油化工有限公司', '中油鼎德润石化（大连）有限公司', '中海中化石化（大连）有限公司', '中海卉宏石油化工（大连）有限公司', '中